In [17]:

#!sudo pip3 install requests
#!pip3 install tqdm
#!pip3 install matplotlib
#!pip3 install pyyaml
#!pip3 install torchvision
#!pip3 install scipy
#!pip3 install pandas


  Using cached pandas-2.0.3-cp311-cp311-macosx_11_0_arm64.whl (10.7 MB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)


In [1]:

# load libraries


import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import sys
import numpy as np
import torch

#plot a cv2 image
import cv2


In [3]:
# add modified YOLOv8 library (cloned from git: URL)

sys.path.append('ultralytics')
from ultralytics import YOLO #make sure you do not have another library with the same name

In [4]:
# This block defines some useful functions

# define a function that reads the labels from the label file
def labelreader(labpath=None):
        labels = []
        with open(labpath, 'r') as f:
            for line in f:
                #append each line to the labels as floats
                labels.append(line.strip())

        for i in range(len(labels)):
            labels[i] = labels[i].split(' ')
            for j in range(len(labels[i])):
                labels[i][j] = float(labels[i][j])

        #cast each element to float and store to tensor
        labels = torch.tensor(labels)
        return labels


#This function generates an image where the CAM heatmap has been overlayed onto the original image
def make_heatmap(image, heatmap, transparancy = 0.3, plot=False):
    heatmapp = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    
    imgg = image.astype(np.uint8)
    cam_img = transparancy * heatmapp + (1-transparancy) * imgg
    cam_img = cam_img.astype(np.uint8)
    
    if plot:
        cv2.imshow("image",cam_img)
    
    return cam_img



# This function plots the heatmap and the bounding boxes on the same image
def plot_heatmap(image, heatmap, bboxes = None, transparancy = 0.3, savename = None, boxlabels = None):
    
    cam_img = make_heatmap(image, heatmap, transparancy, plot=False)
    
    if bboxes is not None:
        colors = [[0, 0, 255], [0, 255, 0], [255, 0, 0], [255,255,255]]
        if boxlabels is None:
            boxlabels = ["sediment", "agglutinated", "calcareous", "planktic"]
        
        labheight = -10
        labwidth = -7
        imgsz = (image.shape[0],image.shape[1])
        for i in range(bboxes.shape[0]):
            box = bboxes[i,:]
            cls = box[0]
            cls = int(cls)
            xc = float(box[1])
            yc = float(box[2])
            w = float(box[3])
            h = float(box[4])
            x1 = round((xc-0.5*w)*imgsz[0])
            x2 = round((xc+0.5*w)*imgsz[0])
            y1 = round((yc-0.5*h)*imgsz[1])
            y2 = round((yc+0.5*h)*imgsz[1])
            
            cv2.rectangle(cam_img, (x1, y1), (x2, y2), colors[cls], 2)
            cv2.putText(cam_img, boxlabels[cls], (x1+labwidth, y1+labheight),cv2.FONT_HERSHEY_SIMPLEX, 0.8, colors[cls], 2)
    
    if savename is not None:
        cv2.imwrite(savename, cam_img)
    cv2.imshow("image",cam_img)
    
    return cam_img



In [5]:
model = YOLO("yolov8x.pt")
catimg = Image.open("cats.jpg")

#remember to add label file path in "./ultralytics/ultralytics/yolo/engine/predictor.py, line 382"
results = model.predict(source=catimg, save=False, visualize = True, imgsz=1280)  # save plotted images


/Users/emy016/Dropbox/Postdoc2/Kurs/noratest/noratest-root/ultralytics/ultralytics/yolo/engine/predictor.py:142: RuntimeWarning: invalid value encountered in divide
  cam = cam / cam.max()
/Users/emy016/Dropbox/Postdoc2/Kurs/noratest/noratest-root/ultralytics/ultralytics/yolo/engine/predictor.py:148: RuntimeWarning: invalid value encountered in cast
  heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
0: 1280x1280 2 cats, 12595.4ms
Speed: 15.5ms preprocess, 12595.4ms inference, 7.8ms postprocess per image at shape (1, 3, 1280, 1280)


In [9]:
# lets reload the image as a cv2 image
image = cv2.imread("cats.jpg")
#image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#give the path to the label file of the sample image
labpath = "cats.txt"

#load labels into useable format
bboxes = labelreader(labpath)

In [10]:
# Lets import the heatmap from the 

run = 15 #run number for the prediction

layers = [18,19,20,21] #which layer to use for the heatmap

resultfolder = "./runs/detect/"+ "predict" + str(run) + "/"

for layer in layers:

    #load heatmap
    heatmap = np.load(resultfolder + str(layer) + ".npy")

    #run the function
    savename = "catimg"+str(run) + "-layer" + str(layer) +"-heatmap.jpg"
    cam_img = plot_heatmap(image,heatmap, bboxes, transparancy = 0.3, savename = savename, boxlabels = ["cat"])


In [ ]:
#Load pre-trained model
model = YOLO("forams4-1280-augmentedraw-bigtest-200epochs-yolov8x.pt")

#load example image
im0 = Image.open("sampleimg-0.jpg")
im1 = Image.open("sampleimg-1.jpg")
im2 = Image.open("sampleimg-2.jpg")
im3 = Image.open("sampleimg-3.jpg")


#remember to add label file path in "./ultralytics/ultralytics/yolo/engine/predictor.py, line 382"
#run prediction with visualize = True to generate heatmap
results = model.predict(source=im0, save=False, visualize = True)  # save plotted images

#The results are stored in /runs/detect/predict...

In [10]:
# lets reload the image as a cv2 image
image = cv2.imread("sampleimg-3.jpg")

#give the path to the label file of the sample image
labpath = "sampleimg-3.txt"

#load labels into useable format
bboxes = labelreader(labpath)

In [11]:
# Lets import the heatmap from the 

run = 4 #run number for the prediction


layer = 21 #which layer to use for the heatmap

resultfolder = "./runs/detect/"+ "predict" + str(run) + "/"

#load heatmap
heatmap = np.load(resultfolder + str(layer) + ".npy")


#run the function
savename = "sampleimg"+str(run)+"-heatmap.jpg"
cam_img = plot_heatmap(image,heatmap, bboxes, transparancy = 0.3, savename = savename)
